In [30]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [38]:
Discrete(3).sample()

2

In [41]:
Box(0,1,shape=(3,)).sample()

array([0.46122116, 0.19521162, 0.21260057], dtype=float32)

In [42]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([12.043642], dtype=float32))])

In [49]:
MultiBinary(6).sample()

array([1, 1, 0, 1, 0, 0], dtype=int8)

In [53]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 0])

## Building our Environment

In [54]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low= 0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60

    def step(self, action):
        self.state += action -1
        self.shower_length -= 1

        if self.state >=37 and self.state <=39:
            reward =1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render(self):

        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)

        self.shower_length = 60
        return self.state

In [57]:
env=ShowerEnv()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [58]:
env.observation_space.sample()

array([60.908154], dtype=float32)

In [59]:
env.reset()

array([39.])

In [62]:
# from stable_baselines3.common.env_checker import check_env

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
# check_env(env, warn=True)

##testing

In [64]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-38
Episode:2 Score:-12
Episode:3 Score:-60
Episode:4 Score:-18
Episode:5 Score:-58


In [65]:
env.close()

##train

In [ ]:
# log_path = os.path.join('Training', 'Logs')
# if you want to save the logs

In [66]:
model = PPO("MlpPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [67]:
model.learn(total_timesteps=100000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.7    |
| time/              |          |
|    fps             | 497      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -36.4       |
| time/                   |             |
|    fps                  | 396         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010655312 |
|    clip_fraction        | 0.0813      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000273   |
|    learning_rate        | 0.

In [68]:

evaluate_policy(model, env, n_eval_episodes=10, render=True)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(59.2, 0.9797958971132712)